In [1]:
!git clone https://github.com/irsafilo/KION_DATASET.git
!pip install pytorch-lightning
!unzip KION_DATASET/interactions.csv.zip
!unzip KION_DATASET/data_original.zip


Cloning into 'KION_DATASET'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 129 (delta 66), reused 10 (delta 1), pack-reused 14 (from 1)
Receiving objects: 100% (129/129), 194.05 MiB | 10.65 MiB/s, done.
Resolving deltas: 100% (67/67), done.
Updating files: 100% (9/9), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 51.2 MB/s eta 0:00:00
Archive:  KION_DATASET/interactions.csv.zip
  inflating: interactions.csv        
Archive:  KION_DATASET/data_original.zip
   creating: data_original/
  inflating: data_original/interactions.csv  
  inflating: __MACOSX/data_original/._interactions.csv  
  inflating: data_original/users.csv  
  inflating: __MACOSX/data_original/._users.csv  
  inflating: data_original/items.csv  
  inflating: __MACOSX/data_original/._items.csv  


# Intro

Всем привет!
В этой лекции мы разберемся как обучить простую DSSM-нейронку на данных KION, а еще поскорим ее в оффлайне. Погнали!

In [2]:
import ast
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from random import randint, random
from scipy.sparse import coo_matrix, hstack
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances, cosine_similarity
import torch
from pathlib import Path
from tqdm import tqdm

Загрузим данные Киона

In [3]:
KION_FOLDER = Path('data_original')

users_df = pd.read_csv(KION_FOLDER / 'users.csv')
items_df = pd.read_csv(KION_FOLDER / 'items.csv')
interactions_df = pd.read_csv(KION_FOLDER / 'interactions.csv')

# Подготовка данных

## EDA

Для начала посмотрим на данные:

In [4]:
users_df.head()

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0


In [5]:
items_df.head()

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."


In [6]:
interactions_df.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0


## Собираем матрицу взаимодействий



In [7]:
interactions_df.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0


В датасете взаимодействий есть непопулярные фильмы и малоактивные пользователи. Кроме того, в таблице взаимодействий есть события с низким качеством взаимодействия - когда юзер начал смотреть фильм, но вскоре после начала просмотра выключил.

Отфильтруем такие события*, малоактивных юзеров и непопулярные фильмы.

_Можете не фильтровать такие события, тогда у вас будет больше негативных примеров._

In [8]:
interactions_df.item_id.value_counts()

,count
item_id,
10440,202457
15297,193123
9728,132865
13865,122119
4151,91167
...,...
8451,1
1164,1
6414,1


In [9]:
interactions_df.user_id.value_counts()

,count
user_id,
416206,1341
1010539,764
555233,685
11526,676
409259,625
...,...
911142,1
1009040,1
1007900,1


In [10]:
print(f"N users before: {interactions_df.user_id.nunique()}")
print(f"N items before: {interactions_df.item_id.nunique()}\n")

# отфильтруем все события взаимодействий, в которых пользователь посмотрел
# фильм менее чем на 35 процентов
interactions_df = interactions_df[interactions_df.watched_pct > 35]

# соберем всех пользователей, которые посмотрели
# больше 10 фильмов (можете выбрать другой порог)
valid_users = []

c = Counter(interactions_df.user_id)
for user_id, entries in c.most_common():
  if entries > 10:
    valid_users.append(user_id)

# и соберем все фильмы, которые посмотрели больше 10 пользователей
valid_items = []

c = Counter(interactions_df.item_id)
for item_id, entries in c.most_common():
  if entries > 10:
    valid_items.append(item_id)

# отбросим непопулярные фильмы и неактивных юзеров
interactions_df = interactions_df[interactions_df.user_id.isin(valid_users)]
interactions_df = interactions_df[interactions_df.item_id.isin(valid_items)]

print(f"N users after: {interactions_df.user_id.nunique()}")
print(f"N items after: {interactions_df.item_id.nunique()}")

N users before: 962179
N items before: 15706

N users after: 54213
N items after: 6140


После фильтрации может получиться так, что некоторые айтемы/юзеры есть в датасете взаимодействий, но при этом они отсутствуют в датасетах айтемов/юзеров или наоборот. Поэтому найдем id айтемов и id юзеров, которые есть во всех датасетах и оставим только их.

In [11]:
common_users = set(interactions_df.user_id.unique()).intersection(set(users_df.user_id.unique()))
common_items = set(interactions_df.item_id.unique()).intersection(set(items_df.item_id.unique()))

print(len(common_users))
print(len(common_items))

interactions_df = interactions_df[interactions_df.item_id.isin(common_items)]
interactions_df = interactions_df[interactions_df.user_id.isin(common_users)]

items_df_filtered = items_df[items_df.item_id.isin(interactions_df['item_id'].unique())].copy()
users_df_filtered = users_df[users_df.user_id.isin(interactions_df['user_id'].unique())].copy()

44959
6140


In [12]:
item_cat_feats = ['content_type', 'release_year',
                  'for_kids', 'age_rating',
                  'studios', 'countries', 'directors']

for col in item_cat_feats:
  items_df_filtered[col] = items_df_filtered[col].fillna('unknown')
  items_df_filtered[f'{col}_encoded'] = items_df_filtered[col].astype('category').cat.codes

items_df_filtered

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,...,actors,description,keywords,content_type_encoded,release_year_encoded,for_kids_encoded,age_rating_encoded,studios_encoded,countries_encoded,directors_encoded
8,9853,film,Лабиринты прошлого,Todos lo saben,2018.0,"криминал, детективы, драмы, зарубежные, триллеры",Испания,unknown,16.0,unknown,...,"Барбара Ленни, Инма Куэста, Карла Кампра, Пене...","Испанка Лаура, давно переехавшая в Буэнос-Айре...","Лабиринты, прошлого, 2018, Испания, семейные, ...",0,88,2,3,24,201,447
10,3526,film,Код «Красный»,Red Joan,2018.0,"биография, экранизации, драмы, зарубежные, мел...",Великобритания,unknown,18.0,unknown,...,"Бен Майлз, Джуди Денч, Лоуренс Спэллман, Софи ...",Тихая английская пенсионерка Джоан попадает по...,"Код, Красный, 2018, Великобритания, друзья, лю...",0,88,2,4,24,39,3534
16,2904,film,Первая ведьма,The Wretched,2019.0,"зарубежные, мистика, ужасы",США,unknown,18.0,unknown,...,"Ази Тесфай, Блейн Крокарелл, Габриэла Квезада ...",Подросток Бен замечает пугающие перемены в пов...,"Первая, ведьма, 2019, США, каникулы, отцы, дет...",0,89,2,4,24,322,575
20,2635,film,Рождество трех медведей,The Three Bears and the Perfect Gift,2019.0,"фильмы, для детей, сказки, зарубежные, семейное",США,unknown,6.0,unknown,...,"Джеймс Престон, Дженни МакШейн, Джозефин Булло...","Когда-то Мать Природа, создавшая зачарованный ...","Рождество, трех, медведей, 2019, США, друзья, ...",0,89,2,1,24,322,4
38,1622,film,Тайное окно,Secret Window,2004.0,"триллеры, детективы",США,unknown,12.0,unknown,...,"Джонни Депп, Джон Туртурро, Мария Белло, Тимот...",Писатель Морт Рейни живет в доме на берегу. Ещ...,"алкоголь, измена, детектив, река Миссисипи, ре...",0,74,2,2,24,322,1429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15956,1325,series,Старое ружьё,Staroye ruzh'ye,2014.0,"драмы, военные",Россия,unknown,16.0,unknown,...,"Тимофей Трибунцев, Ирина Пегова, Добрыня Белев...",1941 год. Петр – простой учитель в маленькой д...,"вторая мировая война, человек с ружьём",1,84,2,3,24,294,1747
15957,15610,series,Цена любви,Tsena Lubvi,2015.0,мелодрамы,Россия,unknown,16.0,unknown,...,"Анна Невская, Юрий Батурин, Анастасия Матвеева...",Преподаватель университета Елена однажды пришл...,NaN,1,85,2,3,24,294,12
15958,6443,series,Полярный круг,Arctic Circle,2018.0,"драмы, триллеры, криминал","Финляндия, Германия",unknown,16.0,unknown,...,"Иина Куустонен, Максимилиан Брюкнер, Пихла Вии...","Во время погони за браконьерами по лесу, сотру...","убийство, вирус, расследование преступления, н...",1,88,2,3,24,462,3699
15959,2367,series,Надежда,NaN,2020.0,"драмы, боевики",Россия,0.0,18.0,unknown,...,"Виктория Исакова, Александр Кузьмин, Алексей М...",Оригинальный киносериал от создателей «Бывших»...,"Надежда, 2020, Россия",1,90,0,4,24,294,1552


In [13]:
user_cat_feats = ["age", "income", "sex", "kids_flg"]

for col in user_cat_feats:
  users_df_filtered[col] = users_df_filtered[col].fillna('unknown')
  users_df_filtered[f'{col}_encoded'] = users_df_filtered[col].astype('category').cat.codes

users_df_filtered

,user_id,age,income,sex,kids_flg,age_encoded,income_encoded,sex_encoded,kids_flg_encoded
24,269408,age_35_44,income_20_40,Ж,1,2,2,1,1
27,384532,age_25_34,income_40_60,М,1,1,3,2,1
66,216495,age_25_34,income_20_40,М,0,1,2,2,0
81,515668,age_25_34,income_20_40,М,0,1,2,2,0
136,824452,age_65_inf,income_20_40,М,0,5,2,2,0
...,...,...,...,...,...,...,...,...,...
840084,1029836,age_65_inf,income_40_60,М,0,5,3,2,0
840098,605991,age_35_44,income_20_40,Ж,0,2,2,1,0
840099,925679,age_35_44,income_20_40,М,1,2,2,2,1
840137,486585,age_25_34,income_20_40,Ж,0,1,2,1,0


In [14]:
interactions_df

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
14,5324,8437,2021-04-18,6598,92.0
18,927973,9617,2021-06-19,8422,100.0
20,896751,8081,2021-05-17,6358,100.0
...,...,...,...,...,...
5476218,840031,429,2021-04-10,7307,100.0
5476224,663351,2301,2021-07-25,9337,100.0
5476226,435089,13475,2021-07-06,4624,85.0
5476239,610017,7107,2021-05-10,1133,75.0


In [15]:
interactions_df['item_id'].nunique()

6136

Соберем взаимодействия в матрицу user*item так, чтобы в строках этой матрицы были user_id, в столбцах - item_id, а на пересечениях строк и столбцов - единица, если пользователь взаимодействовал с айтемом и ноль, если нет.

Такую матрицу удобно собирать в numpy array, однако нужно помнить, что numpy array индексируется порядковыми индексами, а нам же удобнее использовать item_id и user_id.

Создадим некие внутренние индексы для user_id и item_id - uid и iid. Для этого просто соберем все user_id и item_id и пронумеруем их по порядку.

In [16]:
interactions_df["uid"] = interactions_df["user_id"].astype("category")
interactions_df["uid"] = interactions_df["uid"].cat.codes

interactions_df["iid"] = interactions_df["item_id"].astype("category")
interactions_df["iid"] = interactions_df["iid"].cat.codes

print(sorted(interactions_df.iid.unique())[:5])
print(sorted(interactions_df.uid.unique())[:5])
interactions_df.head()

[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]


,user_id,item_id,last_watch_dt,total_dur,watched_pct,uid,iid
0,176549,9506,2021-05-11,4250,72.0,7234,3500
1,699317,1659,2021-05-29,8317,100.0,28657,594
14,5324,8437,2021-04-18,6598,92.0,208,3084
18,927973,9617,2021-06-19,8422,100.0,37990,3536
20,896751,8081,2021-05-17,6358,100.0,36735,2953


In [17]:
print(interactions_df.item_id.nunique())
print(items_df_filtered.item_id.nunique())
print(interactions_df.user_id.nunique())
print(users_df_filtered.user_id.nunique())

set(items_df_filtered.item_id.unique()) - set(interactions_df.item_id.unique())

6136
6136
44959
44959


set()

Для того, чтобы можно было удобно превратить iid/uid в item_id/user_id и наоборот соберем словари

{iid: item_id}, {uid: user_id} и {item_id: iid}, {user_id: uid}.

In [18]:
iid_to_item_id = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("iid").to_dict()["item_id"]
item_id_to_iid = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("item_id").to_dict()["iid"]

uid_to_user_id = interactions_df[["uid", "user_id"]].drop_duplicates().set_index("uid").to_dict()["user_id"]
user_id_to_uid = interactions_df[["uid", "user_id"]].drop_duplicates().set_index("user_id").to_dict()["uid"]

И проиндексируем датасеты users_ohe_df и items_ohe_df по внутренним айди:

In [19]:
items_df_filtered["iid"] = items_df_filtered["item_id"].apply(lambda x: item_id_to_iid[x])
items_df_filtered = items_df_filtered.set_index("iid")

users_df_filtered["uid"] = users_df_filtered["user_id"].apply(lambda x: user_id_to_uid[x])
users_df_filtered = users_df_filtered.set_index("uid")

In [20]:
items_df_filtered.head(4)

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,...,actors,description,keywords,content_type_encoded,release_year_encoded,for_kids_encoded,age_rating_encoded,studios_encoded,countries_encoded,directors_encoded
iid,,,,,,,,,,,,,,,,,,,,,
3619,9853,film,Лабиринты прошлого,Todos lo saben,2018.0,"криминал, детективы, драмы, зарубежные, триллеры",Испания,unknown,16.0,unknown,...,"Барбара Ленни, Инма Куэста, Карла Кампра, Пене...","Испанка Лаура, давно переехавшая в Буэнос-Айре...","Лабиринты, прошлого, 2018, Испания, семейные, ...",0,88,2,3,24,201,447
1268,3526,film,Код «Красный»,Red Joan,2018.0,"биография, экранизации, драмы, зарубежные, мел...",Великобритания,unknown,18.0,unknown,...,"Бен Майлз, Джуди Денч, Лоуренс Спэллман, Софи ...",Тихая английская пенсионерка Джоан попадает по...,"Код, Красный, 2018, Великобритания, друзья, лю...",0,88,2,4,24,39,3534
1023,2904,film,Первая ведьма,The Wretched,2019.0,"зарубежные, мистика, ужасы",США,unknown,18.0,unknown,...,"Ази Тесфай, Блейн Крокарелл, Габриэла Квезада ...",Подросток Бен замечает пугающие перемены в пов...,"Первая, ведьма, 2019, США, каникулы, отцы, дет...",0,89,2,4,24,322,575
934,2635,film,Рождество трех медведей,The Three Bears and the Perfect Gift,2019.0,"фильмы, для детей, сказки, зарубежные, семейное",США,unknown,6.0,unknown,...,"Джеймс Престон, Дженни МакШейн, Джозефин Булло...","Когда-то Мать Природа, создавшая зачарованный ...","Рождество, трех, медведей, 2019, США, друзья, ...",0,89,2,1,24,322,4


In [21]:
users_df_filtered.head(4)

,user_id,age,income,sex,kids_flg,age_encoded,income_encoded,sex_encoded,kids_flg_encoded
uid,,,,,,,,,
11047,269408,age_35_44,income_20_40,Ж,1,2,2,1,1
15756,384532,age_25_34,income_40_60,М,1,1,3,2,1
8852,216495,age_25_34,income_20_40,М,0,1,2,2,0
21134,515668,age_25_34,income_20_40,М,0,1,2,2,0


# DSSM starter's pack

Что нужно для того, чтобы собрать плюс-минус любую нейронку?
- функция потерь
- dataloader
- код архитектуры нейронной сети



Начнем с данных:

In [22]:
from torch.utils.data import Dataset, DataLoader

SEED = 42

class TupleDataset(Dataset):
  def __init__(self,
               user_pos_pairs: np.ndarray,
               user_features: pd.DataFrame,
               item_features: pd.DataFrame,
               n_negatives: int = 1) -> None:
    self.user_pos_pairs = user_pos_pairs
    self.user_features = user_features
    self.item_features = item_features
    self.all_items = item_features.index.values
    self.rng = np.random.default_rng(SEED)
    self.n_negatives = n_negatives

  def __len__(self):
    return len(self.user_pos_pairs)


  def __getitem__(self, index):
    user, pos = self.user_pos_pairs[index]
    negative = self.rng.choice(self.all_items, size=self.n_negatives).item()

    user_features = self.user_features.loc[user].to_dict()
    pos_features = self.item_features.loc[pos].to_dict()
    neg_features = self.item_features.loc[negative].to_dict()

    return {
        'user_features': user_features,
        'pos_features': pos_features,
        'neg_features': neg_features
    }

In [23]:
from pytorch_lightning import LightningDataModule

class DssmDataModule(LightningDataModule):
  def __init__(self, train_ds: TupleDataset, train_batch_size: int = 1):
    super().__init__()
    self.train_ds = train_ds
    self.train_batch_size = train_batch_size

  def train_dataloader(self):
    return DataLoader(self.train_ds, batch_size=self.train_batch_size)

In [24]:
user_feature_cols = [f'{col}_encoded' for col in user_cat_feats]
item_feature_cols = [f'{col}_encoded' for col in item_cat_feats]
user_features = users_df_filtered[user_feature_cols]
item_features = items_df_filtered[item_feature_cols]

In [25]:
user_features

,age_encoded,income_encoded,sex_encoded,kids_flg_encoded
uid,,,,
11047,2,2,1,1
15756,1,3,2,1
8852,1,2,2,0
21134,1,2,2,0
33752,5,2,2,0
...,...,...,...,...
42157,5,3,2,0
24833,2,2,1,0
37907,2,2,2,1


In [26]:
item_features

,content_type_encoded,release_year_encoded,for_kids_encoded,age_rating_encoded,studios_encoded,countries_encoded,directors_encoded
iid,,,,,,,
3619,0,88,2,3,24,201,447
1268,0,88,2,4,24,39,3534
1023,0,89,2,4,24,322,575
934,0,89,2,1,24,322,4
580,0,74,2,2,24,322,1429
...,...,...,...,...,...,...,...
475,1,84,2,3,24,294,1747
5807,1,85,2,3,24,294,12
2328,1,88,2,3,24,462,3699


In [27]:
pairs = interactions_df[['uid', 'iid']].values
pairs

array([[ 7234,  3500],
       [28657,   594],
       [  208,  3084],
       ...,
       [17861,  4980],
       [24986,  2581],
       [15738,  6017]], dtype=int32)

In [28]:
train_ds = TupleDataset(user_pos_pairs=pairs, user_features=user_features, item_features=item_features)

In [29]:
train_ds[0]

{'user_features': {'age_encoded': 2,
  'income_encoded': 3,
  'sex_encoded': 2,
  'kids_flg_encoded': 0},
 'pos_features': {'content_type_encoded': 0,
  'release_year_encoded': 83,
  'for_kids_encoded': 2,
  'age_rating_encoded': 0,
  'studios_encoded': 24,
  'countries_encoded': 322,
  'directors_encoded': 1946},
 'neg_features': {'content_type_encoded': 1,
  'release_year_encoded': 87,
  'for_kids_encoded': 2,
  'age_rating_encoded': 2,
  'studios_encoded': 24,
  'countries_encoded': 459,
  'directors_encoded': 1626}}

In [30]:
batch_size = 64
dm = DssmDataModule(train_ds=train_ds, train_batch_size=batch_size)

In [31]:
next(iter(dm.train_dataloader()))

{'user_features': {'age_encoded': tensor([2, 2, 0, 1, 2, 2, 2, 2, 1, 1, 4, 1, 1, 3, 2, 1, 1, 4, 3, 2, 2, 2, 2, 1,
          2, 0, 0, 2, 2, 4, 1, 2, 2, 3, 2, 2, 0, 3, 2, 3, 2, 3, 2, 2, 2, 1, 2, 0,
          1, 2, 2, 1, 2, 1, 2, 2, 0, 2, 0, 0, 1, 2, 1, 2]),
  'income_encoded': tensor([3, 3, 2, 2, 3, 3, 2, 3, 4, 2, 2, 3, 2, 2, 3, 2, 2, 3, 2, 2, 5, 3, 5, 2,
          0, 2, 2, 3, 2, 2, 3, 3, 2, 2, 3, 2, 2, 2, 3, 2, 2, 3, 2, 5, 3, 2, 2, 3,
          3, 2, 5, 2, 2, 4, 3, 3, 3, 2, 2, 2, 2, 2, 3, 2]),
  'sex_encoded': tensor([2, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2,
          2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2,
          2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1]),
  'kids_flg_encoded': tensor([0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
          1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1,
          1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0])},
 'pos_features': {

## Собираем модель

Для того, чтобы обучить модель, нам нужно получить три вектора - вектор юзера, вектор "хорошего" айтема и вектор (один или несколько) "плохого" айтема. Для этого нам нужно две модели ("хороший" и "плохой" айтем будут семплироваться одной и той же моделью).  

Модель юзера будет иметь два входа:
- вход для фичей юзера (фичи из users_features)
- вход для вектора айтемов, которые посмотрел юзер (строка interactions_vec, которая соответствует uid конкретного юзера)

Выход модели юзера будет размерностью N_FACTORS.

У модели айтема будет один вход для фичей айтема (из items_features) и один выход также размерностью N_FACTORS.

Общая архитектура будет вот такой:
- есть модель юзера и модель айтема
- обе модели семплируют юзер и айтем-фичи во внутреннее пространство размерностью N_FACTORS
- модель айтема семплирует два айтема - "хороший" и "плохой"
- в итоге получается три вектора размерностью N_FACTORS (вектор юзера, вектор "хорошего" айтема и вектор "плохого" айтема)
- затем полученные векторы конкатенируются, по ним считается loss
- profit

Для того, чтобы собрать модель, помимо размерности внутреннего пространства, нам нужно знать еще размерность вектора юзера и вектора айтема. Зададим их сразу.

In [32]:
N_FACTORS = 64

CAT_EMBEDDING_DIM = 16

# в датасетах есть столбец user_id/item_id, помним, что он не является фичей для обучения!
ITEM_MODEL_SHAPE = len(item_feature_cols) * CAT_EMBEDDING_DIM
USER_META_MODEL_SHAPE = len(user_feature_cols) * CAT_EMBEDDING_DIM

# USER_INTERACTION_MODEL_SHAPE = (interactions_vec.shape[1], )

print(f"N_FACTORS: {N_FACTORS}")
print(f"ITEM_MODEL_SHAPE: {ITEM_MODEL_SHAPE}")
print(f"USER_META_MODEL_SHAPE: {USER_META_MODEL_SHAPE}")
# print(f"USER_INTERACTION_MODEL_SHAPE: {USER_INTERACTION_MODEL_SHAPE}")

N_FACTORS: 64
ITEM_MODEL_SHAPE: 112
USER_META_MODEL_SHAPE: 64


In [33]:
import torch.nn as nn


class DssmTower(nn.Module):
  def __init__(self, feat_vocab_sizes: dict[str, int], cat_feat_emb_dim: int, hidden_dim: int, out_dim: int):
    super().__init__()
    self.embedding = nn.ModuleDict({
        cat_feat: nn.Embedding(cat_feat_vocab_size, cat_feat_emb_dim) for cat_feat, cat_feat_vocab_size in feat_vocab_sizes.items()
    })
    self.features = list(feat_vocab_sizes.keys())
    self.layer_1 = nn.Linear(cat_feat_emb_dim * len(feat_vocab_sizes), hidden_dim)
    self.layer_2 = nn.Linear(hidden_dim, hidden_dim)
    self.layer_3 = nn.Linear(hidden_dim, out_dim)

  def forward(self, batch):
    embeddings = []
    for feature in self.features:
      feature_embedding = self.embedding[feature](batch[feature])
      embeddings.append(feature_embedding)
    embedding = torch.cat(embeddings, dim=1)

    layer_1 = self.layer_1(embedding)
    layer_2 = self.layer_2(layer_1)
    layer_2 += layer_1
    output = self.layer_3(layer_2)
    return output


In [34]:
user_feat_vocab_size = dict()
for col in user_features.columns:
  user_feat_vocab_size[col] = user_features[col].nunique()

item_feat_vocab_size = dict()
for col in item_features.columns:
  item_feat_vocab_size[col] = item_features[col].nunique()

In [35]:
user_feat_vocab_size

{'age_encoded': 7,
 'income_encoded': 7,
 'sex_encoded': 3,
 'kids_flg_encoded': 2}

In [36]:
item_feat_vocab_size

{'content_type_encoded': 2,
 'release_year_encoded': 92,
 'for_kids_encoded': 3,
 'age_rating_encoded': 6,
 'studios_encoded': 28,
 'countries_encoded': 535,
 'directors_encoded': 4041}

In [37]:
batch = next(iter(dm.train_dataloader()))
batch

{'user_features': {'age_encoded': tensor([2, 2, 0, 1, 2, 2, 2, 2, 1, 1, 4, 1, 1, 3, 2, 1, 1, 4, 3, 2, 2, 2, 2, 1,
          2, 0, 0, 2, 2, 4, 1, 2, 2, 3, 2, 2, 0, 3, 2, 3, 2, 3, 2, 2, 2, 1, 2, 0,
          1, 2, 2, 1, 2, 1, 2, 2, 0, 2, 0, 0, 1, 2, 1, 2]),
  'income_encoded': tensor([3, 3, 2, 2, 3, 3, 2, 3, 4, 2, 2, 3, 2, 2, 3, 2, 2, 3, 2, 2, 5, 3, 5, 2,
          0, 2, 2, 3, 2, 2, 3, 3, 2, 2, 3, 2, 2, 2, 3, 2, 2, 3, 2, 5, 3, 2, 2, 3,
          3, 2, 5, 2, 2, 4, 3, 3, 3, 2, 2, 2, 2, 2, 3, 2]),
  'sex_encoded': tensor([2, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2,
          2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2,
          2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1]),
  'kids_flg_encoded': tensor([0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
          1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1,
          1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0])},
 'pos_features': {

In [38]:
user_tower = DssmTower(
    feat_vocab_sizes=user_feat_vocab_size,
    cat_feat_emb_dim=CAT_EMBEDDING_DIM,
    hidden_dim=64,
    out_dim=64
)
item_tower = DssmTower(
    feat_vocab_sizes=item_feat_vocab_size,
    cat_feat_emb_dim=CAT_EMBEDDING_DIM,
    hidden_dim=64,
    out_dim=64
)

In [39]:
batch['neg_features']

{'content_type_encoded': tensor([0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
         1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]),
 'release_year_encoded': tensor([69, 86, 85, 80, 77, 88, 88, 89, 72, 82, 80, 87, 89, 84, 89, 90, 84, 90,
         75, 83, 87, 88, 62, 90, 82, 89, 84, 83, 90, 90, 84, 62, 84, 75, 81, 75,
         88, 84, 54, 91, 81, 84, 89, 91, 81, 87, 90, 86, 90, 85, 89, 75, 81, 89,
         81, 88, 85, 90, 64, 91, 88, 74, 80, 85]),
 'for_kids_encoded': tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
 'age_rating_encoded': tensor([3, 3, 4, 3, 3, 3, 2, 4, 3, 3, 2, 3, 4, 0, 3, 4, 2, 4, 2, 3, 3, 0, 1, 2,
         4, 3, 0, 2, 3, 4, 4, 3, 3, 3, 3, 2, 3, 0, 1, 4, 3, 3, 4, 1, 3, 4, 2, 4,
         3, 

In [40]:
item_tower.embedding

ModuleDict(
  (content_type_encoded): Embedding(2, 16)
  (release_year_encoded): Embedding(92, 16)
  (for_kids_encoded): Embedding(3, 16)
  (age_rating_encoded): Embedding(6, 16)
  (studios_encoded): Embedding(28, 16)
  (countries_encoded): Embedding(535, 16)
  (directors_encoded): Embedding(4041, 16)
)

In [41]:
with torch.no_grad():
  print(user_tower(batch['user_features']).size(), item_tower(batch['pos_features']).size())

torch.Size([64, 64]) torch.Size([64, 64])


In [42]:
from pytorch_lightning import LightningModule
from torch.optim import Adam


class DssmLitModule(LightningModule):
  def __init__(self, user_tower: DssmTower, item_tower: DssmTower, optim_hparams: dict):
    super().__init__()
    self.user_tower = user_tower
    self.item_tower = item_tower
    self.optim_hparams = optim_hparams
    self.loss = torch.nn.BCEWithLogitsLoss()

  def training_step(self, batch, batch_idx):
    user_feats = batch['user_features']
    pos_feats = batch['pos_features']
    neg_feats = batch['neg_features']

    user_embs = self.user_tower(user_feats) # batch_size x out_dim
    pos_embs = self.item_tower(pos_feats) # batch_size x out_dim
    neg_embs = self.item_tower(neg_feats) # batch_size x out_dim

    item_embs = torch.cat((pos_embs.unsqueeze(1), neg_embs.unsqueeze(1)), dim=1) # batch_size x 2 x out_dim

    dot_scores = (user_embs.unsqueeze(1) @ item_embs.transpose(1, 2)).squeeze()
    labels = torch.zeros_like(dot_scores)
    labels[:, 0] = 1

    loss = self.loss(dot_scores, labels)

    self.log('train_loss', loss.item(), on_epoch=True, on_step=True, prog_bar=True)

    return loss


  def configure_optimizers(self):
    return Adam(self.parameters(), **self.optim_hparams)

In [43]:
user_tower = DssmTower(
    feat_vocab_sizes=user_feat_vocab_size,
    cat_feat_emb_dim=CAT_EMBEDDING_DIM,
    hidden_dim=64,
    out_dim=64
)
item_tower = DssmTower(
    feat_vocab_sizes=item_feat_vocab_size,
    cat_feat_emb_dim=CAT_EMBEDDING_DIM,
    hidden_dim=64,
    out_dim=64
)
dssm_module = DssmLitModule(user_tower, item_tower, optim_hparams={'lr': 1e-3})

In [44]:
dssm_module

DssmLitModule(
  (user_tower): DssmTower(
    (embedding): ModuleDict(
      (age_encoded): Embedding(7, 16)
      (income_encoded): Embedding(7, 16)
      (sex_encoded): Embedding(3, 16)
      (kids_flg_encoded): Embedding(2, 16)
    )
    (layer_1): Linear(in_features=64, out_features=64, bias=True)
    (layer_2): Linear(in_features=64, out_features=64, bias=True)
    (layer_3): Linear(in_features=64, out_features=64, bias=True)
  )
  (item_tower): DssmTower(
    (embedding): ModuleDict(
      (content_type_encoded): Embedding(2, 16)
      (release_year_encoded): Embedding(92, 16)
      (for_kids_encoded): Embedding(3, 16)
      (age_rating_encoded): Embedding(6, 16)
      (studios_encoded): Embedding(28, 16)
      (countries_encoded): Embedding(535, 16)
      (directors_encoded): Embedding(4041, 16)
    )
    (layer_1): Linear(in_features=112, out_features=64, bias=True)
    (layer_2): Linear(in_features=64, out_features=64, bias=True)
    (layer_3): Linear(in_features=64, out_featu

In [45]:
with torch.no_grad():
  print(dssm_module.training_step(batch, 0))

tensor(0.7400)


Определяем трейнер:

In [46]:
from pytorch_lightning import Trainer


trainer = Trainer(max_epochs=1, enable_checkpointing=False)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [64]:
trainer.fit(dssm_module, dm)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type              | Params | Mode 
---------------------------------------------------------
0 | user_tower | DssmTower         | 12.8 K | train
1 | item_tower | DssmTower         | 90.9 K | train
2 | loss       | BCEWithLogitsLoss | 0      | train
---------------------------------------------------------
103 K     Trainable params
0         Non-trainable params
103 K     Total params
0.415     Total estimated model params size (MB)
22        Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

# Inference

Отлично! Мы подготовили данные, собрали модель по архитектуре DSSM и обучили ее.
Теперь возьмем случайного юзера и случайный айтем. Как понять, насколько этот айтем релевантен юзеру?

Нужно:
- получить вектор айтема;
- получить вектор юзера;
- посчитать расстояние между ними.

Это расстояние и есть мера релевантности.


In [47]:
# берем рандомного юзера
rand_uid = np.random.choice(list(user_features.index))

# получаем фичи юзера и вектор его просмотров айтемов
rand_uid_feats = user_features.loc[rand_uid].to_dict()

# берем рандомный айтем
rand_iid = np.random.choice(list(item_features.index))
# получаем фичи айтема
rand_iid_feats = item_features.loc[rand_iid].to_dict()


In [49]:
for key in rand_uid_feats:
  rand_uid_feats[key] = torch.tensor([rand_uid_feats[key]])
for key in rand_iid_feats:
  rand_iid_feats[key] = torch.tensor([rand_iid_feats[key]])


In [54]:
(dssm_module.user_tower(rand_uid_feats) * dssm_module.item_tower(rand_iid_feats)).sum()

tensor(-0.1857, grad_fn=<SumBackward0>)